In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist
import numpy as np

In [ ]:
batch_size = 100
original_dim = 28*28
latent_dim = 2
intermediate_dim = 256
n_epoch = 5
epsilon_std = 1.0

In [ ]:
def sampling(args: tuple):
    z_mean, z__log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [ ]:
x = Input(shape=(original_dim,), name='input')
h = Dense(intermediate_dim, activation='relu', name='encoding')(x)
z_mean = Dense(latent_dim, name='mean')(h)
z_log_var = Dense(latent_dim, name='log-variance')(h)
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
encoder = Model(x, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

In [ ]:
input_decoder = Input(shape=(latent_dim,),name='input_decoder')
decoder_h = Dense(intermediate_dim, activation='relu', name='decoder_h')(input_decoder)
x_decoded = Dense(original_dim, activation='sigmoid', name='flat_decoded')(decoder_h)
decoder = Model(input_decoder, x_decoded, name='decoder')

In [ ]:
output_combined = decoder(encoder(x)[2])
vae = Model(x, output_combined)
vae.summary()

In [ ]:
def vae_loss(x, x_decoded_mean, z_log_var, z_mean, original_dim=original_dim):
    xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
    kl_loss = -0.5 * K.sum(1+z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss

In [ ]:
vae.compile(optimizer='rmsprop', loss=vae_loss)